In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
import pandas as pd
import regex as re
import csv
from itertools import islice
import pickle
import numpy as np
import json
import os
import sys
import argparse
from pathlib import Path
import pprint
import tempfile
import re
from typing import Dict, Text

import sklearn as sk
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import reuters
from nltk.data import find

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import ndcg_score
from sklearn.metrics import average_precision_score

In [ ]:
!pip install pydot --quiet
!pip install gensim==3.8.3 --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text --quiet
!pip install transformers --quiet
!pip install pydot --quiet

     |████████████████████████████████| 24.2 MB 1.2 MB/s 
     |████████████████████████████████| 4.6 MB 4.3 MB/s 
     |████████████████████████████████| 511.7 MB 5.5 kB/s 
     |████████████████████████████████| 5.8 MB 43.7 MB/s 
     |████████████████████████████████| 1.6 MB 54.8 MB/s 
     |████████████████████████████████| 438 kB 61.4 MB/s 
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 57.7 MB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
     |████████████████████████████████| 101 kB 11.4 MB/s 


In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q scann

     |████████████████████████████████| 88 kB 3.2 MB/s 
     |████████████████████████████████| 11.2 MB 3.9 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text
import tensorflow_recommenders as tfrs

from transformers import BertTokenizer, TFBertModel

In [ ]:
%cd /gdrive/MyDrive/nlp-yuan_code/FinBERT-QA
from src.process_data import *

/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
RO_FiQA_DATA_PATH = '/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/'
RO_FIQA_INDEX = "/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/retriever/lucene-index-fiqa/"
WR_PICKLE_DATA = '/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/'
WR_PICKLE_TRANSIENT_DATA = '/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/transient/'
WR_INTERIM_DATA = '/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/interim/'

In [ ]:
train_set = load_pickle(WR_INTERIM_DATA + "train_set.pickle")
test_set = load_pickle(WR_INTERIM_DATA + "test_set.pickle")
qid_to_text = load_pickle(WR_INTERIM_DATA + "qid_to_text.pickle")
docid_to_text = load_pickle(WR_INTERIM_DATA + "docid_to_text.pickle")
text_to_docid = load_pickle(WR_INTERIM_DATA + "text_to_docid.pickle")

In [ ]:
def generate_ids(data):
  qids = []
  aids = []
  qa_pairs = []


  #for i, seq in enumerate(tqdm(train_set)):
  for i, seq in enumerate(data):
    qid, ans_labels, cands = seq[0], seq[1], seq[2]
    #print (qid, ans_labels, cands)
    

    # For each answer in the candidates
    for docid in ans_labels:
      qids.append(qid)
      aids.append(docid)
      qa_pairs.append((qid, docid))

  unique_qids = np.unique(qids)
  unique_aids = np.unique(aids)
  return unique_qids, unique_aids, qa_pairs

data = []
data.extend(train_set)
data.extend(test_set)
      
qids, aids, qa_pairs = generate_ids(data)

In [ ]:
unique_questions = np.unique([qid_to_text[qid] for qid in qids])
unique_answers = np.unique([docid_to_text[aid] for aid in aids])

def get_two_towers(embedding_size):
  question_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_questions, mask_token=None),
    # We add an additional embedding to account for unknown tokens.
    tf.keras.layers.Embedding(len(qids) + 1, embedding_dimension)
  ])

  answer_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_answers, mask_token=None),
    # We add an additional embedding to account for unknown tokens.
    tf.keras.layers.Embedding(len(aids) + 1, embedding_dimension)
  ])
  return question_model, answer_model

In [ ]:
class QAModel(tfrs.Model):

  def __init__(self, question_model, answer_model):
    super().__init__()
    self.question_model: tf.keras.Model = question_model
    self.answer_model: tf.keras.Model = answer_model
    metrics = tfrs.metrics.FactorizedTopK(
        candidates=answers.batch(128).map(answer_model)
        )
    task = tfrs.tasks.Retrieval(
        metrics=metrics
        )
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    q_embeddings = self.question_model(features["question"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    a_embeddings = self.answer_model(features["answer"])

    # The task computes the loss and the metrics.
    return self.task(q_embeddings, a_embeddings)

In [ ]:
def run_twotower_scores(data, scann_index):
  ap = []
  rr = []
  cg = []
  skipcnt = 0

  for j, seq in enumerate(data):
    qid, ans_labels, _ , _ = seq[0], seq[1], seq[2], seq[3]
    score, cand_ans = scann_index(tf.constant([qid_to_text[qid]]))
    cands = []
    cands_score = []

    for cand_anss, cand_score in zip(cand_ans[0], np.ravel(score).tolist()):
      cands.append(text_to_docid.get(cand_anss.numpy().decode('utf-8')))
      cands_score.append(cand_score)


    max_width = -1
    rr_ = 0
    ap_ = 0.0
    precision_ = 0.0
    relcnt_ = 0

    top_k = 10

    for i in range(len(cands)):
      docid = cands[i]
      if docid in ans_labels and rr_ == 0:
        rr_ = 1/(i+1)
    
    relscores = [1 if docid in ans_labels else 0 for docid in cands]
    pos = [1.0/(i+1) for i in range(len(cands))]
    ap_ = average_precision_score(relscores,pos) if sum(relscores) != 0 else 0 

    mcnt = max(len(cands), len(ans_labels))
    m = dict([(ai, 1/(i+1)) for i, ai in enumerate(ans_labels)])

    if len(ans_labels)==0 or len(cands)==0:
      cg_ = 0.0
    elif len(ans_labels)==1:
      cg_ = 1.0 if len(cands)>0 and cands[0] in ans_labels else 0
    elif len(cands) == 1:
      cg_ = 1.0 if len(ans_labels) and cands[0] in ans_labels else 0
    else:
      relscores = np.asarray([[ m[cands[i]] if i<len(cands) and m.get(cands[i]) else 0 for i in range(mcnt)]])
      pos = np.asarray([[1/(i+1) for i in range(mcnt)]])
      cg_ = ndcg_score(relscores, pos)  

    ap.append(ap_)
    rr.append(rr_)
    cg.append(cg_)
  return rr, ap, cg

In [ ]:
print(len(qids), len(aids), len(qa_pairs))
print(len(unique_questions), len(unique_answers), len(qa_pairs))

questions = tf.data.Dataset.from_tensor_slices([qid_to_text[qid] if isinstance(qid_to_text.get(qid),str) else '' for qid, aid in qa_pairs])
answers = tf.data.Dataset.from_tensor_slices([docid_to_text[aid] if isinstance(docid_to_text.get(aid),str) else '' for qid, aid in qa_pairs])

len(answers), len(questions)

6648 17110 17110
6648 17071 17110


(17110, 17110)

In [ ]:
ds = tf.data.Dataset.zip((questions, answers))
ds = ds.map(lambda x, y : {"question": x, "answer": y})

tf.random.set_seed(42)
shuffled = ds.shuffle(17_110, seed=42, reshuffle_each_iteration=False)

In [ ]:
cached_train = shuffled.shuffle(17_110).batch(1300)
cached_test =  shuffled.take(856).batch(150)

In [ ]:
embedding_dimension = 64
question_model, answer_model = get_two_towers(embedding_dimension)
model = QAModel(question_model, answer_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
model.fit(cached_train, epochs=5)

Epoch 1/5
14/14 [==============================] - 17s 854ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 5.8445e-05 - factorized_top_k/top_10_categorical_accuracy: 2.9223e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0015 - factorized_top_k/top_100_categorical_accuracy: 0.0037 - loss: 8228.1250 - regularization_loss: 0.0000e+00 - total_loss: 8228.1250
Epoch 2/5
14/14 [==============================] - 12s 812ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.3088 - factorized_top_k/top_10_categorical_accuracy: 0.4425 - factorized_top_k/top_50_categorical_accuracy: 0.7279 - factorized_top_k/top_100_categorical_accuracy: 0.8434 - loss: 8180.8672 - regularization_loss: 0.0000e+00 - total_loss: 8180.8672
Epoch 3/5
14/14 [==============================] - 12s 806ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_cat

In [ ]:
model.evaluate(cached_test, return_dict=True)

6/6 [==============================] - 2s 217ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2231 - factorized_top_k/top_5_categorical_accuracy: 0.8329 - factorized_top_k/top_10_categorical_accuracy: 0.9381 - factorized_top_k/top_50_categorical_accuracy: 0.9988 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 447.0235 - regularization_loss: 0.0000e+00 - total_loss: 447.0235


{'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.9380841255187988,
 'factorized_top_k/top_1_categorical_accuracy': 0.22313083708286285,
 'factorized_top_k/top_50_categorical_accuracy': 0.9988317489624023,
 'factorized_top_k/top_5_categorical_accuracy': 0.8329439163208008,
 'loss': 332.2534484863281,
 'regularization_loss': 0,
 'total_loss': 332.2534484863281}

In [ ]:
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.question_model)
scann_index.index_from_dataset(
  tf.data.Dataset.zip((answers.batch(100), answers.batch(100).map(model.answer_model)))
)

In [ ]:
# Get recommendations.
scores, cands = scann_index(tf.constant([qid_to_text[5]]))

print(f'QUESTION: ', qid_to_text[5])
print(f"Recommendations for user 42: {cands[0, :10]}")

QUESTION:  Starting a new online business
Recommendations for user 42: [b"A practical issue is that insider trading transfers wealth from most investors to the few insiders. If this were permitted, non-insiders would rarely make any money, and they'd stop investing. That would then defeat the purpose of the capital markets which is to attract capital. A moral issue is that managers and operators of a company should act in shareholders' interests. Insider trading directly takes money from other shareholders and transfers it to the insider. It's a nasty conflict of interest (and would allow any CEO of a public company to make ton of money quickly, regardless of their job performance). In short, shareholders and management should succeed or suffer together, so their interests are as aligned as possible and managers have the proper incentives."
 b"I'd worry more about falsifying documents of sale.  No good reason at all to do that. Detecting counterfeit bills is easy if they're all new bil

In [ ]:
rr, ap, cg = run_twotower_scores(test_set, scann_index)

print('Mean Reciprocal Rank (MRR):', np.mean(rr))
print('Mean average Precision (MAP)', np.mean(ap))
print('Normalized Discounted Cumulative Gain (NDCG)', np.mean(cg))

Mean Reciprocal Rank (MRR): 0.568651985318652
Mean average Precision (MAP) 0.568651985318652
Normalized Discounted Cumulative Gain (NDCG) 0.5150760419681824


In [ ]:
embedding_dimension = 128
question_model, answer_model = get_two_towers(embedding_dimension)
model = QAModel(question_model, answer_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
model.fit(cached_train, epochs=5)

model.evaluate(cached_test, return_dict=True)

scann_index = tfrs.layers.factorized_top_k.ScaNN(model.question_model)
scann_index.index_from_dataset(
  tf.data.Dataset.zip((answers.batch(100), answers.batch(100).map(model.answer_model)))
)

rr, ap, cg = run_twotower_scores(test_set, scann_index)

print('Mean Reciprocal Rank (MRR):', np.mean(rr))
print('Mean average Precision (MAP)', np.mean(ap))
print('Normalized Discounted Cumulative Gain (NDCG)', np.mean(cg))

Epoch 1/5
14/14 [==============================] - 14s 895ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 3.5067e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0022 - factorized_top_k/top_100_categorical_accuracy: 0.0049 - loss: 8228.1199 - regularization_loss: 0.0000e+00 - total_loss: 8228.1199
Epoch 2/5
14/14 [==============================] - 12s 872ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.6400 - factorized_top_k/top_10_categorical_accuracy: 0.8074 - factorized_top_k/top_50_categorical_accuracy: 0.9653 - factorized_top_k/top_100_categorical_accuracy: 0.9902 - loss: 8133.9267 - regularization_loss: 0.0000e+00 - total_loss: 8133.9267
Epoch 3/5
14/14 [==============================] - 12s 813ms/step - factorized_top_k/top_1_categorical_accuracy: 1.7534e-04 - factorized_top_k/top_5_cat

In [ ]:
embedding_dimension = 256
question_model, answer_model = get_two_towers(embedding_dimension)
model = QAModel(question_model, answer_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
model.fit(cached_train, epochs=5)

model.evaluate(cached_test, return_dict=True)

scann_index = tfrs.layers.factorized_top_k.ScaNN(model.question_model)
scann_index.index_from_dataset(
  tf.data.Dataset.zip((answers.batch(100), answers.batch(100).map(model.answer_model)))
)

rr, ap, cg = run_twotower_scores(test_set, scann_index)

print('Mean Reciprocal Rank (MRR):', np.mean(rr))
print('Mean average Precision (MAP)', np.mean(ap))
print('Normalized Discounted Cumulative Gain (NDCG)', np.mean(cg))

Epoch 1/5
14/14 [==============================] - 13s 860ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 5.8445e-05 - factorized_top_k/top_10_categorical_accuracy: 2.9223e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0017 - factorized_top_k/top_100_categorical_accuracy: 0.0038 - loss: 8228.1336 - regularization_loss: 0.0000e+00 - total_loss: 8228.1336
Epoch 2/5
14/14 [==============================] - 13s 925ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.7240 - factorized_top_k/top_10_categorical_accuracy: 0.9047 - factorized_top_k/top_50_categorical_accuracy: 0.9943 - factorized_top_k/top_100_categorical_accuracy: 0.9996 - loss: 8039.3240 - regularization_loss: 0.0000e+00 - total_loss: 8039.3240
Epoch 3/5
14/14 [==============================] - 12s 866ms/step - factorized_top_k/top_1_categorical_accuracy: 4.0912e-04 - factorized_top_k/top_5_cat

In [ ]:
embedding_dimension = 512
question_model, answer_model = get_two_towers(embedding_dimension)
model = QAModel(question_model, answer_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
model.fit(cached_train, epochs=5)

model.evaluate(cached_test, return_dict=True)

scann_index = tfrs.layers.factorized_top_k.ScaNN(model.question_model)
scann_index.index_from_dataset(
  tf.data.Dataset.zip((answers.batch(100), answers.batch(100).map(model.answer_model)))
)

rr, ap, cg = run_twotower_scores(test_set, scann_index)

print('Mean Reciprocal Rank (MRR):', np.mean(rr))
print('Mean average Precision (MAP)', np.mean(ap))
print('Normalized Discounted Cumulative Gain (NDCG)', np.mean(cg))

Epoch 1/5
14/14 [==============================] - 15s 996ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 2.3378e-04 - factorized_top_k/top_10_categorical_accuracy: 4.0912e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0022 - factorized_top_k/top_100_categorical_accuracy: 0.0046 - loss: 8228.4975 - regularization_loss: 0.0000e+00 - total_loss: 8228.4975
Epoch 2/5
14/14 [==============================] - 13s 916ms/step - factorized_top_k/top_1_categorical_accuracy: 5.8445e-05 - factorized_top_k/top_5_categorical_accuracy: 0.7475 - factorized_top_k/top_10_categorical_accuracy: 0.9382 - factorized_top_k/top_50_categorical_accuracy: 0.9971 - factorized_top_k/top_100_categorical_accuracy: 0.9999 - loss: 7852.0103 - regularization_loss: 0.0000e+00 - total_loss: 7852.0103
Epoch 3/5
14/14 [==============================] - 13s 917ms/step - factorized_top_k/top_1_categorical_accuracy: 3.5067e-04 - factorized_top_k/top_5_cat

Save the following:
1. save qa_pairs

In [ ]:
save_pickle(WR_INTERIM_DATA + "qa_pairs.pickle", qa_pairs)